# Projeto Final de Ciência dos Dados. ( PkmnID)

## Algoritmo de predição da categoria de Pokémons por meio de suas imagens.
### O algoritmo realiza a extração e a clusterização de features de imagens por meio do método \"Bag of Visual Words\" (BOVW),classifica-as utilizando o método de machine learning \"Random Forest\" e prevê a categoria de Pokémons por meio de novas imagens.

In [ ]:
!pip install opencv-contrib-python
import cv2
from PIL import ImageTk, Image
import os.path
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
import tkinter as tk
import tkinter.filedialog as tkfd
import pickle

models = {}
for k in ['Random Forest', 'KNearestNeighbors', 'Decision Tree']:
    models[k] = pickle.load(open('Assets/files/' + k + '.sav', 'rb'))
vocab = np.load('Assets/files/Bag_of_Visual_Words.npy', allow_pickle = True)
super_clf = pickle.load(open('Assets/files/super_clf.sav', 'rb'))

In [ ]:
def representa(vocab, img):
    kaze = cv2.KAZE_create()
    kp = kaze.detect(img)
    bowdesc = cv2.BOWImgDescriptorExtractor(kaze, cv2.FlannBasedMatcher())
    bowdesc.setVocabulary(vocab)
    return bowdesc.compute(img, kp)

def show_example(path = os.listdir("Testes/Testes/")[0], plot = True):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(img, dsize=(120, 120))
    if plot:
        plt.imshow(img_resized, cmap='gray', vmin=0, vmax=255)
    return representa(vocab, img_resized)

def predict_lower_models(X, models = models):    
    super_matrix = {}
    for model_name, model in models.items():
        super_matrix[model_name] = model.predict_proba(X).tolist()
    return np.concatenate(list(super_matrix.values()), axis=1)

class Application1:
    def __init__(self, master=None):
        self.master = master
        self.widget1 = tk.Frame(self.master)
        self.widget1.pack()
        self.game = tk.Label(self.widget1, text = 'Por favor, selecione uma imagem para ser analisada:')
        self.game["font"] = ("Verdana", "12", "bold")
        self.game.config(anchor=tk.CENTER)
        self.game.pack()
        self.button = tk.Button(self.widget1)
        self.button["text"] = "Escolher"
        self.button["font"] = ("Cambria", "16", "bold")
        self.button["width"] = 24
        self.button["command"] = self.get_image
        self.button.config(anchor=tk.CENTER)
        self.button.pack()
        self.got_img = False
    
    def get_image(self):
        file = tkfd.askopenfilename(title='Choose a file')
        if file != None:
            self.data = file
            self.image1, self.image2, self.image3, self.got_img = create_report(self.data)
            self.master.destroy()

class Application2:
    def __init__(self, master=None):
        self.master = master
        self.widget1 = tk.Frame(self.master)
        self.widget1.pack()
        self.game = tk.Label(self.widget1, text = 'Análise da imagem:')
        self.game["font"] = ("Verdana", "20", "bold")
        self.game.config(anchor=tk.CENTER)
        self.game.pack()


def create_report(path):

    img_path_1 = 'Assets/files/img_teste_original.png'
    img_path_2 = 'Assets/files/img_teste_processed.png'
    img_path_3 = 'Assets/files/img_teste_features.png'
    img = cv2.imread(path)
    img = cv2.resize(img, (213,160))
    cv2.imwrite(img_path_1, img)
    img_res = cv2.cvtColor(cv2.resize(img, (120,120)), cv2.COLOR_BGR2GRAY)
    cv2.imwrite(img_path_2, img_res)
    kaze = cv2.KAZE_create()
    detect = kaze.detect(img_res)
    keypoints = cv2.drawKeypoints(img_res, detect, None)
    cv2.imwrite(img_path_3, keypoints)
    
    return img_path_1, img_path_2, img_path_3, True

### Abaixo armazenamos os modelos numa estrutura que nos será mais acessível.

In [ ]:
def Main(app = None):
    if app:
        app.destroy()     
    ask_file = tk.Tk()
    ask_file.geometry("900x120")
    app_1 = Application1(ask_file)
    ask_file.mainloop()
    print(app_1.data)
    if app_1.got_img:
        report = tk.Tk()
        report.geometry("900x1080")
        app_report = Application2(report)

        app_report.label2 = tk.Label(report, text = "Imagem Original:")
        app_report.label2['font'] = ("Verdana", "14")
        app_report.label2.pack(pady = (30,0))

        img = ImageTk.PhotoImage(Image.open(app_1.image1))
        app_report.orig_img = tk.Label(report, image = img)
        app_report.orig_img.pack()
        
        app_report.label3 = tk.Label(report, text = "Imagem que o modelo lê:")
        app_report.label3['font'] = ("Verdana", "14")
        app_report.label3.pack(pady = (30,0))

        img2 = ImageTk.PhotoImage(Image.open(app_1.image2))
        app_report.modded_img = tk.Label(report, image = img2)
        app_report.modded_img.pack()

        app_report.label4 = tk.Label(report, text = "Features extraídas:")
        app_report.label4['font'] = ("Verdana", "14")
        app_report.label4.pack(pady = (30,0))

        img3 = ImageTk.PhotoImage(Image.open(app_1.image3))
        app_report.features_array = tk.Label(report, image = img3)
        app_report.features_array.pack()

        features = show_example(app_1.data, plot = False)
        app_report.prediction1 = tk.Label(report, text = "Nosso primeiro modelo acredita que seja: %s" % models["Random Forest"].predict(features)[0])
        app_report.prediction1["font"] = ("Verdana", "14")
        app_report.prediction1.pack(pady = (30,0))

        app_report.prediction2 = tk.Label(report, text = "Nosso modelo final acredita que seja: %s" % super_clf.predict(predict_lower_models(features))[0])
        app_report.prediction2['font'] = ("Verdana", "14")
        app_report.prediction2.pack()

        app_report.prediction3 = tk.Label(report, text = "Eaí, acertamos?")
        app_report.prediction3.pack()

        app_report.restart = tk.Button(report, text = 'Escolher novamente', command = lambda: Main(report))
        app_report.restart.pack()
        
        report.mainloop()


Main()